In [1]:
from pynq import allocate
from pynq import Overlay
import numpy as np
import struct
import ctypes

In [2]:
custom_hw = Overlay("matmul3.bit")

In [3]:
custom_hw?

Type:            Overlay
String form:     <pynq.overlay.Overlay object at 0xaf6171f0>
File:            /usr/local/share/pynq-venv/lib/python3.8/site-packages/pynq/overlay.py
Docstring:      
Default documentation for overlay matmul3.bit. The following
attributes are available on this overlay:

IP Blocks
----------
dma_1                : pynq.lib.dma.DMA
dma_2                : pynq.lib.dma.DMA
out_dma              : pynq.lib.dma.DMA
processing_system7_0 : pynq.overlay.DefaultIP

Hierarchies
-----------
None

Interrupts
----------
None

GPIO Outputs
------------
None

Memories
------------
PSDDR                : Memory
Class docstring:
This class keeps track of a single bitstream's state and contents.

The overlay class holds the state of the bitstream and enables run-time
protection of bindings.

Our definition of overlay is: "post-bitstream configurable design".
Hence, this class must expose configurability through content discovery
and runtime protection.

The overlay class exposes th

In [4]:
custom_hw.ip_dict

{'dma_1': {'fullpath': 'dma_1',
  'type': 'xilinx.com:ip:axi_dma:7.1',
  'bdtype': None,
  'state': None,
  'addr_range': 65536,
  'phys_addr': 1077936128,
  'mem_id': 'S_AXI_LITE',
  'memtype': 'REGISTER',
  'gpio': {},
  'interrupts': {},
  'parameters': {'C_S_AXI_LITE_ADDR_WIDTH': '10',
   'C_S_AXI_LITE_DATA_WIDTH': '32',
   'C_DLYTMR_RESOLUTION': '125',
   'C_PRMRY_IS_ACLK_ASYNC': '0',
   'C_ENABLE_MULTI_CHANNEL': '0',
   'C_NUM_MM2S_CHANNELS': '1',
   'C_NUM_S2MM_CHANNELS': '1',
   'C_INCLUDE_SG': '0',
   'C_SG_INCLUDE_STSCNTRL_STRM': '0',
   'C_SG_USE_STSAPP_LENGTH': '0',
   'C_SG_LENGTH_WIDTH': '20',
   'C_M_AXI_SG_ADDR_WIDTH': '32',
   'C_M_AXI_SG_DATA_WIDTH': '32',
   'C_M_AXIS_MM2S_CNTRL_TDATA_WIDTH': '32',
   'C_S_AXIS_S2MM_STS_TDATA_WIDTH': '32',
   'C_MICRO_DMA': '0',
   'C_INCLUDE_MM2S': '1',
   'C_INCLUDE_MM2S_SF': '1',
   'C_MM2S_BURST_SIZE': '16',
   'C_M_AXI_MM2S_ADDR_WIDTH': '32',
   'C_M_AXI_MM2S_DATA_WIDTH': '32',
   'C_M_AXIS_MM2S_TDATA_WIDTH': '32',
   'C_INCLUDE

In [5]:
SIZE=64
abuf = allocate((SIZE*SIZE*2, ) , dtype=np.float32)
bbuf = allocate((SIZE*SIZE*2, ) , dtype=np.float32)
obuf = allocate((SIZE*SIZE, ) , dtype=np.float32)

a = np.random.random_sample((SIZE,SIZE*2,))
b = np.random.random_sample((SIZE*2,SIZE,))

flatten_a = a.flatten()
flatten_b = b.flatten()

np.copyto(abuf, flatten_a)
np.copyto(bbuf, flatten_b)

In [6]:
#mat_hw = custom_hw.matmul_0

In [7]:
#mat_hw.register_map

In [8]:
dma1 = custom_hw.dma_1
dma2 = custom_hw.dma_2
out_dma = custom_hw.out_dma

In [9]:
dma1.sendchannel.transfer(abuf)
dma2.sendchannel.transfer(bbuf)
out_dma.recvchannel.transfer(obuf)


In [10]:
obuf

PynqBuffer([31.993511, 33.43159 , 32.961132, ..., 31.568974, 32.83881 ,
            32.77336 ], dtype=float32)

In [11]:
dma1.sendchannel.wait()
dma2.sendchannel.wait()
out_dma.recvchannel.wait()

In [13]:
#mat_hw.register_map

In [14]:
dma1.register_map

RegisterMap {
  MM2S_DMACR = Register(RS=1, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=1, IRQDelay=0),
  MM2S_DMASR = Register(Halted=0, Idle=1, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlvErr=0, SGDecErr=0, IOC_Irq=1, Dly_Irq=0, Err_Irq=0, IRQThresholdSts=0, IRQDelaySts=0),
  MM2S_CURDESC = Register(Current_Descriptor_Pointer=0),
  MM2S_CURDESC_MSB = Register(Current_Descriptor_Pointer=0),
  MM2S_TAILDESC = Register(Tail_Descriptor_Pointer=0),
  MM2S_TAILDESC_MSB = Register(Tail_Descriptor_Pointer=0),
  MM2S_SA = Register(Source_Address=377815040),
  MM2S_SA_MSB = Register(Source_Address=0),
  MM2S_LENGTH = Register(Length=32768),
  SG_CTL = Register(SG_CACHE=0, SG_USER=0),
  S2MM_DMACR = Register(RS=0, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=0, IRQDelay=0),
  S2MM_DMASR = Register(Halted=0, Idle=0, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SG

In [15]:
np.matmul(a,b)

array([[31.99350414, 33.43158747, 32.96112854, ..., 32.77997778,
        33.01149999, 31.88894609],
       [28.8280676 , 31.04175952, 32.80454935, ..., 30.42552082,
        31.29239299, 30.73330589],
       [32.14695404, 36.20097592, 34.68882412, ..., 34.37060994,
        34.31920621, 32.8138772 ],
       ...,
       [31.94399706, 34.65953103, 34.96278194, ..., 33.29319814,
        34.01613522, 32.55651829],
       [32.07961755, 32.94277918, 33.82915639, ..., 34.08004465,
        35.59201555, 33.12998231],
       [33.89140249, 34.08605997, 33.55119785, ..., 31.56897444,
        32.83880248, 32.77336147]])

In [16]:
np.isclose(np.matmul(a,b).flatten(),obuf)

PynqBuffer([ True,  True,  True, ...,  True,  True,  True])

In [ ]:
%%timeit
np.matmul(a,b)

In [ ]:
%%timeit
dma1.sendchannel.transfer(abuf)
dma2.sendchannel.transfer(bbuf)
out_dma.recvchannel.transfer(obuf)
dma1.sendchannel.wait()
dma2.sendchannel.wait()
out_dma.recvchannel.wait()

<https://support.xilinx.com/s/question/0D52E00006hpfjmSAA/huge-bram-usage-difference-between-vivado-hls-and-vivado?language=en_US>
<https://support.xilinx.com/s/question/0D52E00006iHhIaSAK/vectors-memory-optimization-using-other-resources-instead-of-bram-vivado-hls?language=en_US>  
pragma HLS resource variable=<variable_name> core=RAM_1P_LUTRAM  
<https://support.xilinx.com/s/article/51455?language=en_US>  
<https://focuskr.tistory.com/113>
<https://ubuntu.com/tutorials/create-a-usb-stick-on-windows#1-overview>  
<https://docs.xilinx.com/v/u/2017.4-English/ug1270-vivado-hls-opt-methodology-guide>  
<https://linuxhint.com/install-ubuntu-external-hard-drive/>  


In [ ]:
np.isclose(np.matmul(a,b).flatten(),obuf)

In [26]:
class matmul_overlay(Overlay):
    def __init__(self, bitfile_name, SIZE=64):
        super().__init__(bitfile_name)
        #self.saxpy_hw = self.krnl_saxpy2_0
        self.abuf = allocate((SIZE*SIZE*2, ) , dtype=np.float32)
        self.bbuf = allocate((SIZE*SIZE*2, ) , dtype=np.float32)
        self.obuf = allocate((SIZE*SIZE, ) , dtype=np.float32)
        #interrupt 생략
    
    def python_float_to_uint(self, num):
        return ctypes.c_uint.from_buffer(ctypes.c_float(num)).value
    
    def register_map(self):
        return self.register_map
    
    def ip_dict(self):
        return self.saxpy_hw.ip_dict()
    
    def run(self, x, y):
        np.copyto(self.abuf, x.flatten())
        np.copyto(self.bbuf, y.flatten())
        #self.saxpy_hw.write(saxpy_hw.register_map.size.address, SIZE)
        #self.saxpy_hw.write(saxpy_hw.register_map.a.address, self.python_float_to_uint(a))
        
        self.dma_1.sendchannel.transfer(self.abuf)
        self.dma_2.sendchannel.transfer(self.bbuf)
        self.out_dma.recvchannel.transfer(self.obuf)
        self.dma_1.sendchannel.wait()
        self.dma_2.sendchannel.wait()
        self.out_dma.recvchannel.wait()
            
        return self.obuf
    
    def assert_by_sw(self, x, y):
        xy = np.matmul(x,y)
        xy_hw = self.run(x,y)
        comparison = np.isclose(xy.flatten(),xy_hw)
        for i in comparison:
            assert(i == True)
            
        return True

In [27]:
MATMUL = matmul_overlay("matmul3.bit")
MATMUL.run(a,b)

PynqBuffer([31.993511, 33.43159 , 32.961132, ..., 31.568974, 32.83881 ,
            32.77336 ], dtype=float32)

In [28]:
MATMUL.assert_by_sw(a,b)

True